<a href="https://colab.research.google.com/github/raz0208/Natural-Language-Processing-Practices/blob/main/TopicModelling/EmbeddingsAnalysis_TopicModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modelling

## Semantic Signal Separation

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [9]:
# Read and load dataset
dataset = pd.read_csv('gdb_dataset.csv')

# Show the datasets
### Abstract Embeddings Sample Dataset
print('Node Content:', dataset.shape)
print(dataset)

Node Content: (1000, 4)
     Unnamed: 0                                              title  \
0             0  Phenotypic variability of Niemann-Pick disease...   
1             1  Recurrent hypoglycemia secondary to metformin ...   
2             2  Adaptation of the Ambulatory and Home Care Rec...   
3             3  Multidimensional family therapy in adolescents...   
4             4  Balanced crystalloids versus isotonic saline i...   
..          ...                                                ...   
995         995  Molecular Sex Identification in Dioecious  Hip...   
996         996  Antimicrobial Peptides: Powerful Biorecognitio...   
997         997  Analysis of Metabolites in White Flowers of  M...   
998         998  Improved Cold Tolerance of Mango Fruit with En...   
999         999  Characterization of the Complete Chloroplast G...   

                                              abstract  year  
0    Background Niemann-Pick disease type C (NPC) i...  2018  
1    Back

In [10]:
# Drop first column
dataset = dataset.drop(dataset.columns[[0,1,3]], axis=1)

dataset

,abstract
0,Background Niemann-Pick disease type C (NPC) i...
1,Background Metformin toxicity is well known to...
2,Background Measuring service use and costs is ...
3,Background Substance use and delinquency are c...
4,Objectives Intravenous fluids are one of the m...
...,...
995,The dioecious property of the sea buckthorn ( ...
996,Bacterial infections represent a serious threa...
997,A total of seven phenolics and 44 metabolites ...
998,Red fruits were suggested to be tolerant to co...
